In [4]:
"""
This set of functions help the users resize
"""
import os
import json

import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import detectron2
import detectron2.data.transforms as T
from detectron2.structures import BoxMode
from detectron2.engine.defaults import DefaultPredictor

import labelme
from labelme import LabelFile
from labelme import utils

import PIL
from PIL import Image


** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



In [5]:
def json_to_dict(directory, img_height, img_width):
    """
    This is the wrapper function that converts all json files in the designated 
    directory into requred image height and width, and store the dataset as
    a Detectron2 input type.
    Inputs:
        - directory: the directory path where all training .json files are located
        - img_heght: the expected image height after conversion in unit of pixel
        - img_width: the expected image width after conversion in unit of pixel
    Outputs:
        - dataset_dict: list[{dictionary1}, {dictionary2},...]. Each dictionary
                        contains segmentation information for each image.
        - imgs: list[nparray1, nparray2,...]. Each np array signifies an image.
        - polygonlist: list[list[]]
    """
    # Load only .json files in the directory
    path = directory
    valid_filetype = [".json"]
    imgs = []
    polygon_list = []
    dataset_dict = []
    img_num = 0
    for file in os.listdir(path):
        filename = os.path.splitext(file)[0] # filename without extension
        ext = os.path.splitext(file)[1] # find the extension of the file
        if ext.lower() not in valid_filetype:
            continue
        img, polygons = json_to_arrs(path+file)
        new_img, new_polygons = img_mask_rescale(img, polygons, img_height, img_width)
        img2save = Image.fromarray(new_img)
        img2save.save(filename+".png")
        reshaped_polygons, bb_boxes = polygon_arr_to_list(new_polygons)
        img_dict = to_dict(path, file, img_num, img_height, img_width, reshaped_polygons, bb_boxes)
        dataset_dict.append(img_dict)
        # Append to global lists for secondary outputs
        imgs.append(new_img)
        polygon_list.append(new_polygons)
        img_num = img_num+1
    return dataset_dict, imgs, polygon_list #imgs,polygon_list  # list[{image1},{image2}]

In [176]:
def test_json_to_dict():
    """
    Test json_to_dict()
    """
    dictionary, imgs, polygon_list = json_to_dict("test_data/test_json/", 200, 500)
    assert isinstance(dictionary, list), "Unexpected output type"
    assert isinstance(dictionary[0], dict), "Unexpected output type"
    assert len(imgs)==len(polygon_list), "Image counts doesn't match polygon counts"
    return

def test_show_img_and_mask():
    """
    Test show_img_and_mask()
    """
    imgs = [np.array([[1,1,1],[1,1,1]]),np.array([[2,2,2],[2,2,2]]),np.array([[3,3,3],[3,3,3]])]
    polygon_list = [[np.array([[1,1],[1,1]]),np.array([[2,2],[2,2]])],[],[]]
    assert len(imgs)==len(polygon_list), "Image counts doesn't match polygon counts"
    return

def test_json_to_arrs():
    """
    Test test_json_to_arrs()
    """
    img, poly_for_single_img = json_to_arrs("test_data/test_json/test_json1.json")
    if len(poly_for_single_img[0]) != 0:
        assert len(poly_for_single_img[0][0]) == 2, "Coordinates have the wrong shape"
    return

def test_polygon_arr_to_list():
    """
    Test polygon_arr_to_list()
    """
    test_poly = [np.array([[1,1],[2,2],[3,3]]), np.array([[1,1],[2,2],[3,3]])]
    new_poly, box = polygon_arr_to_list(test_poly)
    assert len(new_poly[0]) == test_poly[0].shape[0]*test_poly[0].shape[1]
    return

def test_img_mask_rescale():
    """
    Test img_mask_rescale()
    """
    # Test with empty mask array
    new_img, new_poly = img_mask_rescale(imgs[0], np.array([]), 300, 200)
    assert not new_poly == True, "Unexpected results for zero mask scenario"
    return
def test_to_dict():
    """
    Test test_to_dict()
    """
    test_dictionary = to_dict("test_data/test_json/","test_json1.json",
                     1,234,567,[[1, 1, 2, 2, 3, 3], [4, 4, 5, 5, 6, 6]],
                     [[1, 1, 3, 3], [4, 4, 6, 6]])
    for key in ['file_name', 'image_id', 'height', 'width', 'annotations']:
        assert key in dictionary.keys(), "Missing dictionary keys"
    return

In [174]:
dictionary = to_dict("test_data/test_json/","test_json1.json",
                     1,234,567,[[1, 1, 2, 2, 3, 3], [4, 4, 5, 5, 6, 6]],
                     [[1, 1, 3, 3], [4, 4, 6, 6]])
for key in ['file_name', 'image_id', 'height', 'width', 'annotations']:
    assert key in dictionary.keys()
#print(dictionary.keys())

In [128]:
img, poly = json_to_arrs("test_data/test_json/test_json1.json")
poly[0].shape
type(poly)

list

In [129]:
polygon = poly[0]
reshaped_poly = polygon.reshape(1,polygon.shape[0]*polygon.shape[1]).tolist()[0]   
len(reshaped_poly)

60

In [166]:
test_poly = [np.array([[1,1],[2,2],[3,3]]), np.array([[4,4],[5,5],[6,6]])]
print(test_poly[0].shape)
p,b = polygon_arr_to_list(test_poly)
b

(3, 2)


[[1, 1, 3, 3], [4, 4, 6, 6]]

In [131]:
p,b = polygon_arr_to_list(poly)
type(p[0])
#p[0]

list

In [161]:
#imgs = [np.array([[1,1,1],[1,1,1]]),np.array([[2,2,2],[2,2,2]]),np.array([[3,3,3],[3,3,3]])]
#polygon_list = [[np.array([[1,1],[1,1]]),np.array([[2,2],[2,2]])],[],[]]
#print(polygon_list[0][0])

new_img, new_poly = img_mask_rescale(imgs[0], np.array([]), 300, 200)
new_poly = [[[],[]],[]]
not new_poly == True
#s = np.array([[],[]],[])
#s.size

True

In [177]:
test_json_to_dict()
test_show_img_and_mask()
test_json_to_arrs()
test_show_img_and_mask()
test_polygon_arr_to_list()
test_img_mask_rescale()
test_to_dict()

In [53]:
#dictionary, imgs, polygon_list = json_to_dict("test_data/test_json/", 200, 500)
img, poly_for_single_img = json_to_arrs("test_data/test_json/test_json1.json")
#print(len(poly_for_single_img[0][0]))
len(poly_for_single_img[0])
#print(img.shape)
#print(imgs[0].shape)
#print(polygon_list[0])

30

In [8]:


def dictionary_fuction_for_detectron2(directory, img_height, img_width):
    """
    This function converts the output of the main wrapper function
    to the format required by Detectron2.
    """
    dictionary, imgs, polygon_list = json_to_dict(directory, img_height, img_width)
    return dictionary

def show_img_and_mask(imgs,polygon_list):
    """
    This function show the images and annotated masks
         for the training dataset.
    Input:
        - list of nd array images
        - list of polygon mask vertex coordinates
    Output:
        - Inline plot with images and masks
    """
    mask = []
    for img_num,polygon in enumerate(polygon_list):
        plt.figure()
        plt.imshow(imgs[img_num],alpha=1)
        for mask_num,item in enumerate(polygon):
            poly = item.tolist()
            poly.append(item[-1].tolist())
            x, y = zip(*poly)
            mask.append(plt.fill(x,y,alpha=0.5,color="r"))
    for index,item in enumerate(mask):
        plt.show(mask[index])
        
def json_to_arrs(json_file):
    """
    Load a single image and its corresponding polygon masks from a jsonfile
    Output the images as an np array and a the polygon masks as a list of
    np arrays

    Inputs:
        - json_file: a json file that contains the image B64 format and
                     the coordinates of the vertices of the polygon masks
    Outputs:
        - img: np array of the image
        - poly_for_single_img: a list of np arrays, each represent one mask
    """
    # Load data from json
    data = json.load(open(json_file))
    # Append images to a list of nd arrays
    img_data = data.get("imageData")
    img = labelme.utils.img_b64_to_arr(img_data) # load image to np array
    # Append mask to a list of polygons
    poly = data.get("shapes")
    poly_for_single_img = []
    for polyitem in poly:
        polygon = np.array(polyitem.get("points"))
        polygon = np.reshape(polygon,(len(polygon),2))
        poly_for_single_img.append(polygon)

    return img, poly_for_single_img

def polygon_arr_to_list(polygons):
    """
    This funtion converts the a list of polygons for single image to 
    Detectron2 accpeted format and find the bounding boxes
    Inputs:
        - polygons: a list of np arrays that contains masks for a single image
    Outputs:
        - reshaped_polys: 
        - bb_boxes: coordinates of the bounding boxes of the masks
    """
    reshaped_polys = []
    bb_boxes =[]
    for ind, polygon in enumerate(polygons):
        reshaped_poly = polygon.reshape(1,polygon.shape[0]*polygon.shape[1]).tolist()[0]            
        # [min_x, min_y, max_x, max_y]
        bb_box = [np.min(reshaped_poly[0::2]),
                np.min(reshaped_poly[1::2]),
                np.max(reshaped_poly[0::2]),
                np.max(reshaped_poly[1::2])]
        reshaped_polys.append(reshaped_poly)
        bb_boxes.append(bb_box)
    return reshaped_polys, bb_boxes

def img_mask_rescale(img, polygons, height, width):
    """
    Rescale np array image and masks to designated height and width.
    Return new img as np array, new maks as a list of np array
    """
    scale = T.ScaleTransform(np.shape(img)[0],np.shape(img)[1], height, width)
    new_img = scale.apply_image(img, "bilinear")
    new_poly = scale.apply_polygons(polygons)
    return new_img, new_poly

def to_dict(path,jsonfile,img_num,height,width,reshaped_polygon_list, bb_box_list):
    """
    Save all information into detectron2 dictionary
    """
    filename = os.path.splitext(jsonfile)[0]
    new_filename = filename+".png" # create new json filename
    single_img_dict = {}
    single_img_dict["file_name"] = new_filename
    single_img_dict["image_id"] = img_num
    single_img_dict["height"] = height 
    single_img_dict["width"] = width
    
    # Initiate list of dict [{instance1},{instance2},{instance3}}]]
    # Each dict corresponds to annotations of one instance in this image
    objs = []            
    for instance,mask in enumerate(reshaped_polygon_list):
        anno = {}
        # Record segmentation as list[list[float]] as required by Detectron2
        # Each list[float] is one instance
        # in the format of [x1, y1, ..., xn, yn] (in unit of pixels).
        anno["segmentation"] = [reshaped_polygon_list[instance]]
        # Record mask bounding box
        anno["bbox"] = bb_box_list[instance] # Bounding box
        anno["bbox_mode"] = BoxMode.XYXY_ABS # Type of bounding box
        # Record category label: there is only one category, so always 0
        anno["category_id"] = 0 
        objs.append(anno)
    single_img_dict["annotations"] = objs # list of dict [{instance1},{instance2},{instance3}}]]
    return single_img_dict # {image1}
